# JustWalk Steps

In [ ]:
from pybehavior.models import DataSet
from pybehavior.tools import Loader, Preprocessor
import pandas as pd
import datetime

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  

In [ ]:
data = pd.read_pickle('data/walk/data7.pickle')
data.tolist()


# JustWalk EMA

In [1]:
import os

os.environ.get('NCEIKEY')